In [1]:
import numpy as np

In [27]:
def nondelay_embedding(x, E=3, n=1, seed=1):
    """
    Args:
        x [n_timesteps, n_dim]: data
        E {int}: embedding dimension
        n {int}: number of embeddings for generating

    Return:
        [n, n_timesteps, E]
    """
    np.random.seed(seed)
    assert x.ndim==2
    _, n_dim = x.shape

    combinations = np.random.randint(n_dim, size=(n,E))
    return x[:, combinations].swapaxes(0,1)

In [30]:
x = np.arange(120).reshape(20,6)
nondelay_embedding(x, E=2, seed=2)

array([[[  0,   5],
        [  6,  11],
        [ 12,  17],
        [ 18,  23],
        [ 24,  29],
        [ 30,  35],
        [ 36,  41],
        [ 42,  47],
        [ 48,  53],
        [ 54,  59],
        [ 60,  65],
        [ 66,  71],
        [ 72,  77],
        [ 78,  83],
        [ 84,  89],
        [ 90,  95],
        [ 96, 101],
        [102, 107],
        [108, 113],
        [114, 119]]])

In [72]:
def delay_embedding(x, E=None, max_delay=5, n=1, seed=1):
    """
    Args:
        x [n_timesteps]: data
        E {int}: embedding dimension
        max_delay {int}: maximum embedding dimension
        n {int}: number of embeddings for generating

    Return:
        [n, n_timesteps - delay, E] or [n_timesteps - delay, E]
    """
    np.random.seed(seed)
    n_timesteps = len(x)

    # only maximum number of delay is determined
    if E is None:
        # binary array for embedding
        # if True, the correspoding delay interval is used.
        results = np.zeros((n,), dtype=object)
        i = 0
        # calculate for each reconstructed coordinates
        while i<n:
            binary = np.random.randint(2, size=(max_delay)).astype(bool)
            if np.count_nonzero(binary)!=0:
                delays = np.where(binary)[0]
                maximum = delays.max() + 1 # because zero is necessary
                result = np.zeros([n_timesteps - maximum, np.count_nonzero(binary) + 1])
                result[:,0] = x[maximum:] # our embedding system is t-tau type
                for j in range(result.shape[1]-1):
                    result[:,j+1] = x[maximum - delays[j] - 1:-delays[j] - 1]
                results[i] = result
                i += 1
    else:
        # embedding dimension is determined
        assert E <= max_delay
        results = np.zeros((n,), dtype=object)
        for i in range(n):
            combinations = np.sort(np.random.choice(max_delay, E-1, False))
            maximum = combinations.max() + 1
            result = np.zeros([n_timesteps - maximum, E])
            result[:,0] = x[maximum:]
            for j in range(result.shape[1]-1):
                result[:,j+1] = x[maximum - combinations[j]:-combinations[j]]
            results[i] = result
    if n==1:
        return results[0]
    else:
        return results

In [73]:
x = np.arange(50)
delay_embedding(x, E=3, max_delay=5, n=2, seed=3)

array([array([[ 5.,  2.,  1.],
       [ 6.,  3.,  2.],
       [ 7.,  4.,  3.],
       [ 8.,  5.,  4.],
       [ 9.,  6.,  5.],
       [10.,  7.,  6.],
       [11.,  8.,  7.],
       [12.,  9.,  8.],
       [13., 10.,  9.],
       [14., 11., 10.],
       [15., 12., 11.],
       [16., 13., 12.],
       [17., 14., 13.],
       [18., 15., 14.],
       [19., 16., 15.],
       [20., 17., 16.],
       [21., 18., 17.],
       [22., 19., 18.],
       [23., 20., 19.],
       [24., 21., 20.],
       [25., 22., 21.],
       [26., 23., 22.],
       [27., 24., 23.],
       [28., 25., 24.],
       [29., 26., 25.],
       [30., 27., 26.],
       [31., 28., 27.],
       [32., 29., 28.],
       [33., 30., 29.],
       [34., 31., 30.],
       [35., 32., 31.],
       [36., 33., 32.],
       [37., 34., 33.],
       [38., 35., 34.],
       [39., 36., 35.],
       [40., 37., 36.],
       [41., 38., 37.],
       [42., 39., 38.],
       [43., 40., 39.],
       [44., 41., 40.],
       [45., 42., 41.],
       [4

In [80]:
def generalized_delay_embedding(x, E=None, max_delay=5, n=1, seed=1):
    """
    Args:
        x [n_timesteps, n_dim]: data
        E {int}: embedding dimension
        max_delay {int}: maximum embedding dimension
        n {int}: number of embeddings for generating

    Return:
        [n, n_timesteps - delay, E] or [n_timesteps - delay, E]
    """
    np.random.seed(seed)
    n_timesteps, n_dim = x.shape

    # only maximum number of delay is determined
    if E is None:
        # binary array for embedding
        # if True, the correspoding delay interval is used.
        results = np.zeros((n,), dtype=object)
        count = 0
        # calculate for each reconstructed coordinates
        while count<n:
            binary = np.random.randint(2, size=(max_delay+1,n_dim)).astype(bool)
            delays, dims = np.where(binary)
            if delays.min()==0 and len(delays)>1:
                maximum = delays.max()
                result = np.zeros([n_timesteps - maximum, len(delays)])
                for j in range(len(delays)):
                    result[:,j] = x[maximum - delays[j]:n_timesteps - delays[j], dims[j]]
                results[count] = result
                count += 1
    else:
        # embedding dimension is determined
        assert E <= max_delay*n_dim
        results = np.zeros((n,), dtype=object)
        count = 0
        while count < n:
            combinations = np.sort(np.random.choice(max_delay*n_dim, E, False))
            dims, delays = combinations//max_delay, combinations%max_delay
            if delays.min()==0:
                maximum = delays.max()
                result = np.zeros([n_timesteps - maximum, E])
                for j in range(E):
                    result[:,j] = x[maximum - delays[j]:n_timesteps - delays[j], dims[j]]
                results[count] = result
                count += 1
    if n==1:
        return results[0]
    else:
        return results

In [82]:
x = np.arange(120).reshape(20, 6)
generalized_delay_embedding(x, E=10, max_delay=3, n=2)

array([array([[  6.,   0.,  13.,   7.,  14.,   8.,   9.,  10.,   4.,  17.],
       [ 12.,   6.,  19.,  13.,  20.,  14.,  15.,  16.,  10.,  23.],
       [ 18.,  12.,  25.,  19.,  26.,  20.,  21.,  22.,  16.,  29.],
       [ 24.,  18.,  31.,  25.,  32.,  26.,  27.,  28.,  22.,  35.],
       [ 30.,  24.,  37.,  31.,  38.,  32.,  33.,  34.,  28.,  41.],
       [ 36.,  30.,  43.,  37.,  44.,  38.,  39.,  40.,  34.,  47.],
       [ 42.,  36.,  49.,  43.,  50.,  44.,  45.,  46.,  40.,  53.],
       [ 48.,  42.,  55.,  49.,  56.,  50.,  51.,  52.,  46.,  59.],
       [ 54.,  48.,  61.,  55.,  62.,  56.,  57.,  58.,  52.,  65.],
       [ 60.,  54.,  67.,  61.,  68.,  62.,  63.,  64.,  58.,  71.],
       [ 66.,  60.,  73.,  67.,  74.,  68.,  69.,  70.,  64.,  77.],
       [ 72.,  66.,  79.,  73.,  80.,  74.,  75.,  76.,  70.,  83.],
       [ 78.,  72.,  85.,  79.,  86.,  80.,  81.,  82.,  76.,  89.],
       [ 84.,  78.,  91.,  85.,  92.,  86.,  87.,  88.,  82.,  95.],
       [ 90.,  84.,  97.,  